In [1]:
#TFM UNIR BIG DATA

In [72]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from collections import Counter
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier 
from sklearn.svm import SVC
import warnings

In [3]:
df=pd.read_csv("/kaggle/input/dataset/preus_tancament_r.csv")

In [40]:
assets = ['AAPL', 'AMS', 'AMZN', 'BBVA', 'CABK', 'CLNX', 'FER', 'GOOGL', 'GSPC', 'IBE', 
 'IBEX', 'ITX', 'IXIC', 'JPM', 'LLY', 'META', 'MSFT', 'NVDA', 'SAN', 'TEF', 'TSLA']

# Inicialitzar DataFrame de treball
df_features = df
df_features['Date'] = pd.to_datetime(df['Date'])

In [41]:

# Per a cada actiu, generem les següents variables:
# - Retorn diari
# - Target (1 si retorn demà és positiu, 0 si no)
# - Lags (retards dels retorns)
# - Mitjana mòbil i desviació típica (rolling)


#for asset in assets:
#    close_col = f'{asset}.Close'
#    volume_col = f'{asset}.Volume'
#
#    if close_col in df.columns and volume_col in df.columns:
#        # Retorn diari
#        df_features[f'{asset}_return'] = df[close_col].pct_change()
#
#        # Variable objectiu: retorn del dia següent
#        df_features[f'{asset}_target'] = (df_features[f'{asset}_return'].shift(-1) > 0).astype(int)
#
#        # Lags
#        df_features[f'{asset}_lag1'] = df_features[f'{asset}_return'].shift(1)
#        df_features[f'{asset}_lag2'] = df_features[f'{asset}_return'].shift(2)
#
#        # Mitjana mòbil i desviació
#        df_features[f'{asset}_ma5'] = df[close_col].rolling(5).mean()
#        df_features[f'{asset}_std5'] = df[close_col].rolling(5).std()
#
#        # Z-score del volum
#        df_features[f'{asset}_volume_z'] = (
#            (df[volume_col] - df[volume_col].rolling(20).mean()) /
#            df[volume_col].rolling(20).std()
#        )
#
## Eliminem només les files amb valors nuls a qualsevol columna creada
#df_clean = df_features.dropna()
#
# Mostrem les dimensions resultants
#print(f"Número total d'actius: {len(assets)}")
#print(f"Dimensions del df_clean: {df_clean.shape}")

Número total d'actius: 21
Dimensions del df_clean: (2, 275)


In [44]:
df_clean_dict = {}
assets_valids = []

for asset in assets:
    relevant_cols = [f'{asset}_return', f'{asset}_target', f'{asset}_lag1',
                     f'{asset}_lag2', f'{asset}_ma5', f'{asset}_std5', f'{asset}_volume_z']

    if all(col in df_features.columns for col in relevant_cols):
        df_asset = df_features[relevant_cols].dropna()
        num_files_valides = len(df_asset)

        if num_files_valides >= 1900:
            df_clean_dict[asset] = df_asset
            assets_valids.append(asset)
            print(f"{asset}: {num_files_valides} files vàlides (✔)")
        else:
            print(f"{asset}: {num_files_valides} files vàlides (✘)")

AAPL: 1987 files vàlides (✔)
AMS: 1987 files vàlides (✔)
AMZN: 1987 files vàlides (✔)
BBVA: 1987 files vàlides (✔)
CABK: 211 files vàlides (✘)
CLNX: 2 files vàlides (✘)
FER: 1457 files vàlides (✘)
GOOGL: 1987 files vàlides (✔)
GSPC: 1987 files vàlides (✔)
IBE: 17 files vàlides (✘)
IBEX: 3227 files vàlides (✔)
ITX: 49 files vàlides (✘)
IXIC: 1987 files vàlides (✔)
JPM: 1987 files vàlides (✔)
LLY: 1987 files vàlides (✔)
META: 1667 files vàlides (✘)
MSFT: 1987 files vàlides (✔)
NVDA: 1987 files vàlides (✔)
SAN: 1987 files vàlides (✔)
TEF: 1987 files vàlides (✔)
TSLA: 1928 files vàlides (✔)


In [56]:


warnings.filterwarnings("ignore")

# Definim el grid de paràmetres
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

# Validació creuada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for asset in assets_valids:
    df_asset = df_clean_dict[asset]
    X = df_asset.drop(columns=[f'{asset}_target'])
    y = df_asset[f'{asset}_target']

    # Divisió en train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42
    )

    # Grid search amb Random Forest
    clf = RandomForestClassifier(random_state=42)
    grid = GridSearchCV(clf, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)

    # Millor model i prediccions
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    # Mètriques
    roc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    results.append({
        'asset': asset,
        'roc_auc': roc,
        'accuracy': acc,
        'conf_matrix': cm,
        'best_params': grid.best_params_
    })

# Mostrem resultats
results_df_rf = pd.DataFrame(results).sort_values(by='roc_auc', ascending=False)
results_df_rf


,asset,roc_auc,accuracy,conf_matrix,best_params
1,AMS,0.589062,0.595477,"[[212, 24], [137, 25]]","{'max_depth': 5, 'min_samples_split': 2, 'n_es..."
13,TEF,0.545337,0.535176,"[[142, 71], [114, 71]]","{'max_depth': None, 'min_samples_split': 5, 'n..."
5,GSPC,0.533105,0.532663,"[[65, 123], [63, 147]]","{'max_depth': 10, 'min_samples_split': 2, 'n_e..."
0,AAPL,0.530486,0.522613,"[[98, 98], [92, 110]]","{'max_depth': 10, 'min_samples_split': 5, 'n_e..."
10,MSFT,0.516313,0.515075,"[[89, 109], [84, 116]]","{'max_depth': 3, 'min_samples_split': 10, 'n_e..."
9,LLY,0.515054,0.494975,"[[73, 123], [78, 124]]","{'max_depth': 3, 'min_samples_split': 2, 'n_es..."
4,GOOGL,0.507387,0.497487,"[[104, 97], [103, 94]]","{'max_depth': 10, 'min_samples_split': 10, 'n_..."
3,BBVA,0.495289,0.507538,"[[154, 56], [140, 48]]","{'max_depth': 5, 'min_samples_split': 2, 'n_es..."
6,IBEX,0.491453,0.509288,"[[80, 234], [83, 249]]","{'max_depth': 3, 'min_samples_split': 2, 'n_es..."
7,IXIC,0.488088,0.525126,"[[67, 120], [69, 142]]","{'max_depth': None, 'min_samples_split': 5, 'n..."


In [60]:
# XGBOOST


results = []

# Paràmetres per fer grid search
param_grid = {
    'max_depth': [3, 5, 10],
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1]
}

for asset in assets_valids:
    X_cols = [col for col in df_features.columns if col.startswith(asset) and col != f'{asset}_target']
    y_col = f'{asset}_target'

    df_model = df_features[X_cols + [y_col]].dropna()
    
    if len(df_model) < 1900:
        continue  # Saltar si no hi ha prou dades

    X = df_model[X_cols]
    y = df_model[y_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    model = XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)

    grid = GridSearchCV(model, param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_prob > 0.5).astype(int)

    roc_auc = roc_auc_score(y_test, y_pred_prob)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results.append({
        'asset': asset,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'conf_matrix': conf_matrix.tolist(),
        'best_params': grid.best_params_
    })

# Convertir resultats a DataFrame
results_df_xgb = pd.DataFrame(results)
results_df_xgb.sort_values(by='roc_auc', ascending=False, inplace=True)
results_df_xgb.reset_index(drop=True, inplace=True)

results_df_xgb.head(10)



,asset,roc_auc,accuracy,conf_matrix,best_params
0,AMS,0.592133,0.592965,"[[298, 56], [187, 56]]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti..."
1,GSPC,0.522571,0.514238,"[[129, 153], [137, 178]]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est..."
2,NVDA,0.516183,0.499162,"[[139, 152], [147, 159]]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est..."
3,GOOGL,0.514339,0.505863,"[[161, 141], [154, 141]]","{'learning_rate': 0.01, 'max_depth': 10, 'n_es..."
4,IXIC,0.508100,0.517588,"[[114, 166], [122, 195]]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est..."
5,AAPL,0.493276,0.489112,"[[140, 155], [150, 152]]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est..."
6,IBEX,0.488428,0.515996,"[[54, 418], [51, 446]]","{'learning_rate': 0.01, 'max_depth': 5, 'n_est..."
7,MSFT,0.487868,0.479062,"[[138, 159], [152, 148]]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti..."
8,TEF,0.487483,0.482412,"[[189, 130], [179, 99]]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est..."
9,BBVA,0.487402,0.499162,"[[215, 99], [200, 83]]","{'learning_rate': 0.01, 'max_depth': 5, 'n_est..."


In [63]:
#Regressión logistica

feature_suffixes = ["_lag1", "_lag2", "_ma5", "_std5", "_volume_z"]
results = []

for asset in assets_valids:
    feature_cols = [f"{asset}{suffix}" for suffix in feature_suffixes]
    target_col = f"{asset}_target"

    # Verifiquem que existeixin totes les columnes necessàries
    if all(col in df_features.columns for col in feature_cols + [target_col]):
        df_model = df_features[feature_cols + [target_col]].dropna()
        X = df_model[feature_cols]
        y = df_model[target_col]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Grid search amb regularització
        param_grid = {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']  # liblinear suporta l1 i l2
        }
        model = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='roc_auc')
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        roc_auc = roc_auc_score(y_test, y_proba)
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        best_params = model.best_params_

        results.append({
            "asset": asset,
            "roc_auc": roc_auc,
            "accuracy": accuracy,
            "conf_matrix": conf_matrix.tolist(),
            "best_params": best_params
        })

results_df_log = pd.DataFrame(results)
results_df_log.sort_values("roc_auc", ascending=False, inplace=True)
results_df_log.reset_index(drop=True, inplace=True)
results_df_log.head(10)


,asset,roc_auc,accuracy,conf_matrix,best_params
0,JPM,0.576748,0.562814,"[[106, 82], [92, 118]]","{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}"
1,TEF,0.535266,0.555276,"[[217, 4], [173, 4]]","{'C': 100, 'penalty': 'l1', 'solver': 'libline..."
2,BBVA,0.534933,0.487437,"[[194, 0], [204, 0]]","{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin..."
3,LLY,0.523665,0.522613,"[[14, 178], [12, 194]]","{'C': 0.01, 'penalty': 'l2', 'solver': 'liblin..."
4,IXIC,0.520530,0.500000,"[[9, 191], [8, 190]]","{'C': 100, 'penalty': 'l1', 'solver': 'libline..."
5,GSPC,0.519736,0.494975,"[[0, 201], [0, 197]]","{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}"
6,MSFT,0.518232,0.500000,"[[0, 199], [0, 199]]","{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin..."
7,NVDA,0.512634,0.512563,"[[0, 194], [0, 204]]","{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin..."
8,GOOGL,0.512563,0.502513,"[[0, 198], [0, 200]]","{'C': 0.1, 'penalty': 'l1', 'solver': 'libline..."
9,AMS,0.503154,0.542714,"[[216, 0], [182, 0]]","{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin..."


In [71]:
# Ensemble de los 3 modelos (RF, XGB, LOG)



ensemble_results = []

for asset in assets_valids:
    X_cols = [col for col in df_features.columns if col.startswith(asset) and col != f'{asset}_target']
    y_col = f'{asset}_target'

    df_model = df_features[X_cols + [y_col]].dropna() 
   
    X = df_model[X_cols]
    y = df_model[y_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Models base
    rf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
    xgb = XGBClassifier(n_estimators=50, max_depth=5, use_label_encoder=False, eval_metric='logloss', random_state=42)
    logreg = LogisticRegression(max_iter=1000, random_state=42)

    # Stacking ensemble
    stack = StackingClassifier(
        estimators=[
            ('rf', rf),
            ('xgb', xgb),
            ('logreg', logreg)
        ],
        final_estimator=LogisticRegression(),
        passthrough=False,
        cv=5
    )

    # Entrenament
    stack.fit(X_train, y_train)
    y_pred = stack.predict(X_test)
    y_proba = stack.predict_proba(X_test)[:, 1]

    # Mètriques
    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba)
    cm = confusion_matrix(y_test, y_pred)

    ensemble_results.append({
        'asset': asset,
        'roc_auc': roc,
        'accuracy': acc,
        'conf_matrix': cm.tolist()
    })

ensemble_results_df = pd.DataFrame(ensemble_results)
ensemble_results_df.sort_values(by='roc_auc', ascending=False, inplace=True)
ensemble_results_df.reset_index(drop=True, inplace=True)
ensemble_results_df.head()





,asset,roc_auc,accuracy,conf_matrix
0,JPM,0.564488,0.515075,"[[161, 27], [166, 44]]"
1,NVDA,0.539393,0.535176,"[[80, 114], [71, 133]]"
2,TEF,0.537490,0.552764,"[[220, 1], [177, 0]]"
3,GSPC,0.523474,0.494975,"[[0, 201], [0, 197]]"
4,GOOGL,0.518535,0.512563,"[[131, 67], [127, 73]]"


In [73]:
#SVM

X_dict = {}
y_dict = {}
X_train_dict = {}
X_test_dict = {}
y_train_dict = {}
y_test_dict = {}


for asset in assets_valids:
    # Variables predictives (són les mateixes que has fet servir abans)
    feature_cols = [
        f'{asset}_lag1', f'{asset}_lag2',
        f'{asset}_ma5', f'{asset}_std5',
        f'{asset}_volume_z'
    ]
    target_col = f'{asset}_target'

    # Comprovem que les columnes existeixen i no tenen valors nuls
    if all(col in df_clean.columns for col in feature_cols + [target_col]):
        df_asset = df_clean[feature_cols + [target_col]].dropna()

        X = df_asset[feature_cols]
        y = df_asset[target_col]

        # Guardem a X_dict i y_dict
        X_dict[asset] = X
        y_dict[asset] = y

        # Split train/test
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=False
        )

        X_train_dict[asset] = X_train
        X_test_dict[asset] = X_test
        y_train_dict[asset] = y_train
        y_test_dict[asset] = y_test

In [ ]:

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

results = {}

for asset in assets_valids:
    df_asset = df_clean_dict[asset]
    
    X = df_asset.drop(columns=[f'{asset}_target'])
    y = df_asset[f'{asset}_target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    print(f"{asset} - Train samples: {len(X_train)}, Test samples: {len(X_test)}")

    if len(X_train) < 3:
        print(f"Skipping {asset} due to too few training samples")
        continue

    grid = GridSearchCV(SVC(probability=True), param_grid, cv=3, scoring='roc_auc')
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

    roc_auc = roc_auc_score(y_test, y_proba)
    accuracy = accuracy_score(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)

    results[asset] = {
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'conf_matrix': conf_mat,
        'best_params': grid.best_params_
    }

AAPL - Train samples: 1589, Test samples: 398
AMS - Train samples: 1589, Test samples: 398
AMZN - Train samples: 1589, Test samples: 398
BBVA - Train samples: 1589, Test samples: 398
GOOGL - Train samples: 1589, Test samples: 398
GSPC - Train samples: 1589, Test samples: 398
